In [ ]:
import os
from pathlib import Path

import syft_rds as sy

import syft_flwr

In [ ]:
DS = "khoa@openmined.org"

!just dump-sim-config {DS}

In [ ]:
PROJECT_DIR = Path(os.getcwd()).parent.parent.absolute()
FLWR_PROJECT_PATH = PROJECT_DIR / "examples" / "embedded-devices"
SIM_DIR = PROJECT_DIR / ".sim"
CONFIG_DIR = SIM_DIR / ".config"

DO1 = "yash@openmined.org"
DO2 = "rasswanth@openmined.org"
DS_CONFIG = f"{CONFIG_DIR}/{DS}.json"

In [ ]:
do_client_1 = sy.init_session(host=DO1, syftbox_client_config_path=DS_CONFIG)
assert do_client_1.host == DO1
assert do_client_1.email == DS
assert not do_client_1.is_admin

do_client_2 = sy.init_session(host=DO2, syftbox_client_config_path=DS_CONFIG)
assert do_client_2.host == DO2
assert do_client_2.email == DS
assert not do_client_2.is_admin

In [ ]:
datasites = [do_client_1, do_client_2]

DATASET_NAME = "fashionmnist"
dataset1 = do_client_1.dataset.get(name=DATASET_NAME)
dataset1.describe()

In [ ]:
dataset2 = do_client_2.dataset.get(name=DATASET_NAME)
dataset2.describe()

## Bootstrap the `flwr` project and run the server side

In [ ]:
flwr_project_path = str(FLWR_PROJECT_PATH)

print(flwr_project_path)

In [ ]:
!syft_flwr bootstrap {flwr_project_path} -a {DS} -d {DO1} -d {DO2}

In [ ]:
!just run-sim-server {flwr_project_path} {DS}

## TODO - Run simulation (locally on DS's machine)


In [ ]:
syft_flwr.run_simulation(
    path=FLWR_PROJECT_PATH,
    mock_paths=[
        do_client.datasets.get(name=DATASET_NAME).mock_path for do_client in datasites
    ],
    num_datasites=len(datasites),
)

## TODO - Submit Jobs

In [ ]:
# Job submission
for client in datasites:
    client.jobs.submit(
        name="MedMNIST FL Experiment",
        description="MedMNIST Federated Learning Experiment",
        user_code_path=FLWR_PROJECT_PATH,
        # runtime="./dockerfile", # path to a local dockerfile
        tags=["federated", "learning", "medical", "mnist"],
    )

syft_flwr.view_server_logs(name=FLWR_PROJECT_PATH.name)